In [6]:
import pickle
import pandas as pd
import numpy as np
from pyspark.sql import DataFrame
from aeon.classification.distance_based import ProximityTree, ProximityForest
import logging

from pyspark.sql import SparkSession
import os
from pyspark.sql import SparkSession
from data_ingestion import DataIngestion
from preprocessing import Preprocessor
from local_model_manager_DT import LocalModelManager_DT
from prediction_manager import PredictionManager
from local_model_manager import LocalModelManager
from evaluation import Evaluator
from utilities import show_compact
import time
import json

In [2]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("ProximityTree") \
    .getOrCreate()

sc = spark.sparkContext

In [4]:
df = spark.read.csv(r'C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\BigDataProject_repo\fulldataset_ECG5000.csv')

In [5]:
df.show(5)

+-----+-----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+------------+------------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+-----------+-----------+-------------+------------+------------+-----------+-----------+-----------+-----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+

In [8]:
current_dir = os.path.gwdir()
project_root = os.path.abspath(os.path.join(current_dir, "..", ".."))

config_DataIngestion = {
            # "data_path": project_root + "/ECG5000/*.tsv" # IF WE DECIDE TO USE .tsv FILES
            "data_path": project_root + "/fulldataset_ECG5000.csv", # IF WE DECIDE TO USE .csv FILES
            "data_percentage": 0.5  #*100% Percentage of data to load for SW development
            }   

ingestion = DataIngestion(spark, )

AttributeError: module 'ntpath' has no attribute 'gwdir'